In [1]:
!pip install -r ../requirements.txt

You should consider upgrading via the 'c:\users\falchetto\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


# Configuration

In [2]:
ROOT_DIR = "../"
DATA_DIR = ROOT_DIR + "data/"

DATASET_2016_10a_FILENAME = DATA_DIR + "RML2016.10a_dict.pkl"

NN_NAME = 'FreeHandV4'

DEFAULT_CONFIG_FILENAME = ROOT_DIR + "default.ini"

In [3]:
import configparser

config = configparser.ConfigParser()
config.read(DEFAULT_CONFIG_FILENAME)

NEURAL_NETWORK_NAME = config.get('NeuralNetworkNames', NN_NAME)
NEURAL_NETWORK_FILENAME = NEURAL_NETWORK_NAME + '.wts.h5'

TRAINING_SET_DIMENSION = config.getfloat('TrainingValues', 'TrainingSetSplitPercentage')
DROPOUT_RATE = config.getfloat('TrainingValues', 'DropoutRate')

# Set up some params 
NB_EPOCH = config.getint('TrainingValues', 'Epochs') # number of epochs to train on
BATCH_SIZE = config.getint('TrainingValues', 'BatchSize') # training batch size

# Imports

In [4]:
import sys
sys.path.append(ROOT_DIR)

import neural_networks
import datasetlib
import traintestsplitlib
import trainlib
import evaluationlib
import dataaugmentationlib

import math

# Dataset

## Dataset in a pandas Dataframe

In [5]:
dataset_df = datasetlib.load_dataset_dataframe(DATASET_2016_10a_FILENAME)

## Raw dataset

In [6]:
dataset_df

,IQ signals,Modulation_Label,SNR
0,"[[-0.0059014712, -0.0023458179, -0.00074506126...",QPSK,2
1,"[[0.0050326153, 0.00094379985, -0.0018932355, ...",QPSK,2
2,"[[0.0052390713, 0.0073890695, 0.007276459, 0.0...",QPSK,2
3,"[[-0.0019859935, -0.0071501383, -0.00527185, -...",QPSK,2
4,"[[0.006674405, 0.0028359746, 0.005630027, 0.00...",QPSK,2
...,...,...,...
219995,"[[0.0062732296, -0.0050519477, 0.006672171, 0....",BPSK,-18
219996,"[[-0.003903181, -0.0015884301, -0.00633375, 2....",BPSK,-18
219997,"[[-0.0105958255, 0.005601244, -0.012161784, 0....",BPSK,-18
219998,"[[-0.002136606, 0.00995837, 0.0059440527, -0.0...",BPSK,-18


# Neural network

## Configuration

In [7]:
config = configparser.ConfigParser()
config.read(DEFAULT_CONFIG_FILENAME)

NEURAL_NETWORK_NAME = config.get('NeuralNetworkNames', NN_NAME)
NEURAL_NETWORK_FILENAME = NEURAL_NETWORK_NAME + '.wts.h5'

# TRAINING_SET_DIMENSION = config.getfloat('TrainingValues', 'TrainingSetSplitPercentage')
TRAINING_SET_DIMENSION = 0.85
DROPOUT_RATE = config.getfloat('TrainingValues', 'DropoutRate')

# Set up some params 
NB_EPOCH = config.getint('TrainingValues', 'Epochs') # number of epochs to train on
BATCH_SIZE = config.getint('TrainingValues', 'BatchSize') # training batch size

## Data prepare

In [8]:
signals = datasetlib.signals(dataset_df)
labels = datasetlib.labels(dataset_df)

# [QPSQ, BPSK, ...]
mods = datasetlib.mods(dataset_df)
# [-18, -16, ...]
snrs = datasetlib.unique_snrs(dataset_df)

### Training and Test data partitioning

In [9]:
n_elements = len(signals)
training_set_percentage = TRAINING_SET_DIMENSION

train_index, test_index = traintestsplitlib.train_test_random_indexes(n_elements, training_set_percentage)

X_train, X_test, Y_train, Y_test = traintestsplitlib.split_x_y_train_test(signals, labels, mods, train_index, test_index)

X_train, Y_train = dataaugmentationlib.rotate_flip_add_gaussian_noise_and_concatenate_with_signals(X_train, Y_train, theta=math.pi/2, standard_deviation=0.0005, increment_percentage=4)

## Build the Neural Network

In [10]:
in_shp = list(X_train.shape[1:])
print(X_train.shape)
print(in_shp)
classes = mods

(935000, 2, 128)
[2, 128]


In [11]:
model = neural_networks.freehand_v4(in_shp)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 128)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 2, 128, 1)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 2, 128, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 2, 128, 16)        272       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 64, 16)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 64, 16)         64        
_________________________________________________________________
dense (Dense)                (None, 1, 64, 256)        4352  

### Train the model

In [ ]:
# perform training ...
#   - call the main training loop in keras for our network+dataset
filepath = NEURAL_NETWORK_FILENAME

history, model = trainlib.train(filepath, model, X_train, Y_train, X_test, Y_test, BATCH_SIZE, NB_EPOCH, validation_split=0)

Epoch 1/100
914/914 [==============================] - 38s 26ms/step - loss: 1.5482
Epoch 2/100
914/914 [==============================] - 23s 25ms/step - loss: 1.2301
Epoch 3/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1821
Epoch 4/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1584  - ETA: 0s - l
Epoch 5/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1452
Epoch 6/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1308
Epoch 7/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1255
Epoch 8/100
914/914 [==============================] - 23s 26ms/step - loss: 1.1180
Epoch 9/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1099
Epoch 10/100
914/914 [==============================] - 23s 25ms/step - loss: 1.1042
Epoch 11/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0988 0s
Epoch 12/100
914/914 [==============================] - 

Epoch 30/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0460
Epoch 31/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0443
Epoch 32/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0446
Epoch 33/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0433 
Epoch 34/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0387
Epoch 35/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0387 0s - loss: 
Epoch 36/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0353
Epoch 37/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0377 3s - - ETA - ETA:
Epoch 38/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0356
Epoch 39/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0357
Epoch 40/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0307
Epoch 41/100
914/914 [============

Epoch 59/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0144
Epoch 60/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0148
Epoch 61/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0117
Epoch 62/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0141
Epoch 63/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0141
Epoch 64/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0093
Epoch 65/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0119
Epoch 66/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0105
Epoch 67/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0087 0s - loss: 1.0
Epoch 68/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0087 0s 
Epoch 69/100
914/914 [==============================] - 23s 26ms/step - loss: 1.0075
Epoch 70/100
914/914 [========================

Epoch 88/100
914/914 [==============================] - 24s 26ms/step - loss: 0.9964
Epoch 89/100
914/914 [==============================] - 23s 25ms/step - loss: 1.0001
Epoch 90/100
914/914 [==============================] - 24s 26ms/step - loss: 0.9964
Epoch 91/100
914/914 [==============================] - 23s 26ms/step - loss: 0.9960
Epoch 92/100
914/914 [==============================] - 26s 28ms/step - loss: 0.9969
Epoch 93/100
914/914 [==============================] - 24s 26ms/step - loss: 0.9948
Epoch 94/100
914/914 [==============================] - 23s 26ms/step - loss: 0.9954
Epoch 95/100
914/914 [==============================] - 31s 34ms/step - loss: 0.9973
Epoch 96/100
914/914 [==============================] - 23s 26ms/step - loss: 0.9946
Epoch 97/100
914/914 [==============================] - 24s 26ms/step - loss: 0.9923 1s - loss: - ETA: 0s -
Epoch 98/100
914/914 [==============================] - 24s 26ms/step - loss: 0.9928
Epoch 99/100
914/914 [====================

# Evaluate and Plot Model Performance

In [ ]:
# Show simple version of performance
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE)
print(score)

## Loss

In [ ]:
evaluationlib.show_loss_curves(history)

## Confusion matrix

In [ ]:
evaluationlib.plot_confusion_matrix(model, X_test, Y_test, BATCH_SIZE, classes)

## Confusion matrix for each SNR

In [ ]:
acc = evaluationlib.plot_confusion_matrix_each_snr(model, snrs, dataset_df, X_test, Y_test, test_index, classes)

## Accuracy performance

In [ ]:
evaluationlib.accuracy_dataframe(acc)

In [ ]:
import statistics

statistics.mean(acc.values())

In [ ]:
evaluationlib.accuracy_curve(snrs, acc, NEURAL_NETWORK_NAME)